# Data Profiling – Superstore Sales Dataset

## Purpose
This notebook performs an initial data profiling to understand
the structure, data types, missing values, inconsistencies and
potential data quality issues in the raw sales dataset.

No cleaning actions are applied in this stage.

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

1️⃣ Load dataset

In [9]:
# Load raw dataset
df = pd.read_csv("../data/raw/superstore_raw.csv")

df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,"42,420.00",South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,"42,420.00",South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,"90,036.00",West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,"33,311.00",South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,"33,311.00",South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.37


2️⃣ Basic overview (shape, info)

In [3]:
df.shape

(9800, 18)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

3️⃣ Column Overview

Below is a brief description of each column in the dataset:

- Row ID: Sequential identifier for each row (likely system-generated).
- Order ID: Unique identifier for each order transaction.
- Order Date: Date when the order was placed.
- Ship Date: Date when the order was shipped.
- Ship Mode: Shipping method used (e.g., Standard Class, Second Class).
- Customer ID: Unique identifier for each customer.
- Customer Name: Full name of the customer.
- Segment: Customer segment classification (e.g., Consumer, Corporate, Home Office).
- Country: Country where the order was placed.
- City: City of the customer.
- State: State of the customer.
- Postal Code: Customer postal code (contains missing values).
- Region: Business region (e.g., West, East, Central, South).
- Product ID: Unique identifier of the product.
- Category: High-level product category.
- Sub-Category: Detailed product classification.
- Product Name: Specific product description.
- Sales: Revenue generated per transaction line.

4️⃣ Missing values

In [5]:
missing = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing / len(df)) * 100

pd.DataFrame({
    "missing_values": missing,
    "missing_percentage": missing_percentage
}).query("missing_values > 0")

,missing_values,missing_percentage
Postal Code,11,0.11


##### Missing Values Observations

- Postal Code contains 11 missing values (~0.11% of the dataset).
- All other columns appear to be complete.

Although the percentage is small, postal codes may be important
for geographic analysis, so further investigation is required
before deciding whether to impute or remove these records.

5️⃣ Duplicates

In [7]:
df.duplicated().sum()

0

In [6]:
df[df.duplicated(keep=False)].sort_values(by="Order ID").head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales


#### Duplicate Records Assessment

A full duplicate check was performed across all columns.

Result:
- No fully duplicated rows were identified in the dataset.

This indicates that each row represents a unique transaction line.

However, it is important to note that multiple rows may still belong
to the same Order ID, since orders can contain multiple products.
This does not represent a data quality issue, but rather the expected
structure of transactional data.

6️⃣ Data Type Validation

In [10]:
df.dtypes

Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code      float64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
dtype: object

#### Data Type Observations

- Order Date and Ship Date are stored as object instead of datetime.
- Postal Code is stored as float64, which is not ideal for a categorical/geographical identifier.
- Most categorical variables are stored as object, requiring consistency validation.

Date columns must be converted to datetime format to enable time-based analysis.
Postal Code may require type correction or formatting adjustments.

7️⃣ Numerical Range Validation

In [11]:
df["Sales"].describe()

count    9,800.00
mean       230.77
std        626.65
min          0.44
25%         17.25
50%         54.49
75%        210.61
max     22,638.48
Name: Sales, dtype: float64

In [12]:
df[df["Sales"] <= 0]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales


#### Sales Column Assessment

The Sales column was reviewed to detect potential anomalies
or invalid values.

Observations:

- The large difference between the mean (230.77) and median (54.49)
  suggests a right-skewed distribution.
- The maximum value (22,638.48) is significantly higher than the
  75th percentile (210.61), indicating potential high-value outliers.
- Since there are no negative or zero values, there is no immediate
  indication of invalid revenue entries.

8️⃣ Date Consistency Check

In [17]:
df_temp = df.copy()
df_temp["Order Date"] = pd.to_datetime(df_temp["Order Date"], format="%d/%m/%Y")
df_temp["Ship Date"] = pd.to_datetime(df_temp["Ship Date"], format="%d/%m/%Y")

(df_temp["Ship Date"] < df_temp["Order Date"]).sum()

0

#### Date Consistency Assessment

A consistency check was performed to verify whether any records
had a Ship Date earlier than the corresponding Order Date.

Result:
- 0 records were found where Ship Date precedes Order Date.

This indicates that the dataset maintains logical temporal consistency
between order placement and shipping events.

Although no inconsistencies were detected, date columns will still be
converted to datetime format in the cleaning stage to enable proper
time-based analysis (e.g., monthly trends, lead time calculation).

9️⃣ Categorical Consistency Check

In [19]:
df["Segment"].value_counts()

Segment
Consumer       5101
Corporate      2953
Home Office    1746
Name: count, dtype: int64

In [20]:
df["Category"].value_counts()

Category
Office Supplies    5909
Furniture          2078
Technology         1813
Name: count, dtype: int64

In [21]:
df["Region"].value_counts()

Region
West       3140
East       2785
Central    2277
South      1598
Name: count, dtype: int64

#### Categorical Variables Assessment

The main categorical variables (Segment, Category, and Region)
were reviewed for inconsistencies or unexpected labels.

Findings:

- Segment contains three consistent groups: Consumer, Corporate, and Home Office.
- Category includes three well-defined groups: Office Supplies, Furniture, and Technology.
- Region is divided into four business regions: West, East, Central, and South.
- No duplicated labels, formatting issues, or unexpected categories were detected.

🔟 Initial Data Quality Findings

An exploratory data quality assessment was conducted to evaluate 
completeness, consistency, and structural integrity of the dataset.

## Summary of Findings

- Missing Values:
  Only 11 missing values were identified in the Postal Code column.
  No other columns contain null values.

- Duplicate Records:
  No duplicate rows were detected.

- Date Consistency:
  No logical inconsistencies were found between Order Date and Ship Date.
  All shipping events occur on or after the order date.

- Categorical Consistency:
  Segment, Category, and Region contain clean and well-defined categories.
  No unexpected labels or formatting issues were detected.

- Numerical Fields:
  No negative or illogical values were found in Sales.
  However, the Sales distribution appears right-skewed,
  which may require further analysis.

## Overall Assessment

The dataset shows a high level of structural integrity and business logic consistency.
Minor cleaning steps will focus on:

- Handling missing Postal Codes
- Converting date columns to datetime format
- Standardizing data types where necessary

The dataset is suitable for further cleaning and preparation for exploratory analysis.